<a href="https://colab.research.google.com/github/AndyCraig200/workout-api/blob/main/metadata_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Collab Stuff
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!

In [ ]:
!pip install ultralytics #To install model in collab
from ultralytics import YOLO
from ultralytics.solutions import ai_gym
import cv2
from PIL import Image
import os
import numpy as np
import re
import pandas as pd

model = YOLO("yolov8n-pose.pt")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

100%|██████████| 6.51M/6.51M [00:00<00:00, 84.7MB/s]


In [ ]:
LEFT_ELBOW_PTS = [5, 7, 9]
RIGHT_ELBOW_PTS = [6, 8, 10]
LEFT_SHOULDER_PTS = [7, 5, 11]
RIGHT_SHOULDER_PTS = [8, 6, 12]
LEFT_HIP_PTS = [5, 11, 13]
RIGHT_HIP_PTS = [6, 12, 14]
LEFT_KNEE_PTS = [11, 13, 15]
RIGHT_KNEE_PTS = [12, 14, 16]
angle_indices = [
    LEFT_ELBOW_PTS,
    RIGHT_ELBOW_PTS,
    LEFT_SHOULDER_PTS,
    RIGHT_SHOULDER_PTS,
    LEFT_HIP_PTS,
    RIGHT_HIP_PTS,
    LEFT_KNEE_PTS,
    RIGHT_KNEE_PTS
]
def estimate_pose_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def calculate_angles(indices, keypoints):
    for ind, k in enumerate(reversed(keypoints)):
      angles = []
      for idx_set in indices:
            p1, p2, p3 = [k[idx] for idx in idx_set]
            angle = estimate_pose_angle(p1, p2, p3)
            angles.append(angle)
      return angles

def separate_string(input_string):
    # Define the regex pattern to extract video, frame, and id
    pattern = r'(\w+)-(\d+)_jpg\.rf\.(\w+)\.jpg'

    # Match the pattern against the input string
    match = re.match(pattern, input_string)

    if match:
        # Extract video, frame, and id from the matched groups
        video = match.group(1)
        frame = match.group(2)
        id = match.group(3)


    else:
        video = "null"
          # Extracting frame number
        frame_match = re.search(r'mov-(\d+)', input_string)
        if frame_match:
            frame = frame_match.group(1)
        else:
            frame = None

        # Extracting ID
        id_match = re.search(r'rf\.([a-f0-9]+)\.jpg', input_string)
        if id_match:
            id = id_match.group(1)
        else:
            id = None

    return video, frame, id

def generate_metadata_csv(classes, dataset_path, metadata_path):
    # Check if the provided path is a directory
    if not os.path.isdir(dataset_path):
        print(f"{dataset_path} is not a valid directory.")
        return
    metadata=[]
    for class_folder in classes:
        class_path = f'{dataset_path}/{class_folder}'
        files = os.listdir(class_path)
        count = 0
        for file_name in files:
            count+=1
            print(f'Processing file {count}/{len(files)} in {class_folder}')
            try:
                video, frame, id = separate_string(file_name)
                img = Image.open(class_path + '/' + file_name)
                results = model.predict(img, verbose=False)
                keypoints = results[0].keypoints.data
                angles = calculate_angles(angle_indices, keypoints)
                metadata.append([video, frame, id, class_folder] + angles)
            except Exception as e:
                # handle other exceptions
                print(f'An error occurred when processing file: {file_name}')

    column_names = ['Video', 'Frame', 'ID','Exercise', 'Left Elbow Angle', 'Right Elbow Angle', 'Left Shoulder Angle', 'Right Shoulder Angle', 'Left Hip Angle', 'Right Hip Angle', 'Left Knee Angle', 'Right Knee Angle']
    df = pd.DataFrame(metadata, columns=column_names)
   # Write DataFrame to CSV file
    df.to_csv(metadata_path, index=False)
def calculate_rebased_keypoints(key_points, bounding_box):
    x1, y1, x2, y2 = bounding_box
    box_width = x2 - x1
    box_height = y2 - y1
    data = [box_width, box_height]

    for kp in key_points:
        x_norm, y_norm = kp
        x_rebased_n = ((x_norm - x1) / box_width)
        y_rebased_n = ((y_norm - y1) / box_height)
        data.extend([x_rebased_n, y_rebased_n])
    return data
def generate_keypoints_metadata_csv(classes, dataset_path, metadata_path):
    # Check if the provided path is a directory
    if not os.path.isdir(dataset_path):
        print(f"{dataset_path} is not a valid directory.")
        return
    metadata=[]
    for class_folder in classes:
        class_path = f'{dataset_path}/{class_folder}'
        files = os.listdir(class_path)
        count = 0
        for file_name in files:
            count+=1
            print(f'Processing file {count}/{len(files)} in {class_folder}')
            try:
                video, frame, id = separate_string(file_name)
                img = Image.open(class_path + '/' + file_name)
                results = model.predict(img, verbose=False)
                row = calculate_rebased_keypoints(key_points=results[0].keypoints.xyn.numpy()[0],
                                          bounding_box=results[0].boxes.xyxyn.numpy()[0])
                metadata.append([video, frame, id, class_folder] + row)
            except Exception as e:
                # handle other exceptions
                print(f'An error occurred when processing file: {file_name}')

    columns = ['Video', 'Frame', 'ID','Exercise','box_width', 'box_height']
    for i in range(17):
        columns.append(f'kp_{i+1}_x')
        columns.append(f'kp_{i+1}_y')
    df = pd.DataFrame(metadata, columns=columns)
   # Write DataFrame to CSV file
    df.to_csv(metadata_path, index=False)


In [ ]:

# Set'folder_path' to the actual path of the folder you want to analyze
classes = ["Deadlift", "Pushup", "Squat", "Other"]
output_path = "/content/drive/MyDrive/capstone/dataset_v2/metadata/metadata_kp_train_v1.csv"
dataset_path = '/content/drive/MyDrive/capstone/dataset_v2/labelling/train/'
generate_keypoints_metadata_csv(classes, dataset_path, output_path)

Streaming output truncated to the last 5000 lines.
Processing file 991/2633 in Pushup
Processing file 992/2633 in Pushup
Processing file 993/2633 in Pushup
Processing file 994/2633 in Pushup
Processing file 995/2633 in Pushup
Processing file 996/2633 in Pushup
Processing file 997/2633 in Pushup
Processing file 998/2633 in Pushup
Processing file 999/2633 in Pushup
Processing file 1000/2633 in Pushup
Processing file 1001/2633 in Pushup
Processing file 1002/2633 in Pushup
Processing file 1003/2633 in Pushup
Processing file 1004/2633 in Pushup
Processing file 1005/2633 in Pushup
Processing file 1006/2633 in Pushup
Processing file 1007/2633 in Pushup
Processing file 1008/2633 in Pushup
Processing file 1009/2633 in Pushup
Processing file 1010/2633 in Pushup
Processing file 1011/2633 in Pushup
Processing file 1012/2633 in Pushup
Processing file 1013/2633 in Pushup
Processing file 1014/2633 in Pushup
Processing file 1015/2633 in Pushup
Processing file 1016/2633 in Pushup
Processing file 1017/2

In [ ]:
len(output[1])

8

In [ ]:
# prompt: Turn a 2d array into a dataframe with 8 columns
def generate_metadata_csv(dataset_path, filename, metadata_path)
import pandas as pd

# Convert the 2D array to a DataFrame
df = pd.DataFrame(output)

# Rename the columns
df.columns = [
    "Left Elbow Angle",
    "Right Elbow Angle",
    "Left Shoulder Angle",
    "Right Shoulder Angle",
    "Left Hip Angle",
    "Right Hip Angle",
    "Left Knee Angle",
    "Right Knee Angle",
]

# Print the DataFrame
print(df)


    Left Elbow Angle  Right Elbow Angle  Left Shoulder Angle  \
0          25.159548          37.457580            47.862926   
1          34.484286          46.667032            36.296629   
2          23.974305          27.536796            37.906252   
3          32.470883          47.118521            32.634815   
4          35.216080          45.670292            35.366043   
5          28.485283          38.405471            29.834635   
6          34.616669          39.716225            35.868383   
7          28.877500          40.373316            32.419793   
8          12.195883          37.181969            47.304394   
9          34.023187          43.809113            34.336932   
10         36.268434          35.646768            50.093284   
11         20.280450          45.051962            32.761173   
12         35.961697          61.490663           124.631599   
13         32.248151          41.063360            32.435059   
14         19.457802          33.163762 

NameError: name 'X_train' is not defined